In [1]:
# from src.pingers import ping_ads_insights_all_platforms
# from src.database.session import db
# from src.s3 import read_csv_from_s3
# from src.models import FacebookAdsInsights, TikTokAdsInsights

ModuleNotFoundError: No module named 'grpc'

In [ ]:
crm = read_csv_from_s3(bucket="lebesgue-crm", path="crm_dataset_dev.csv", add_global_path=False)
crm = crm[["shop_id", "industry"]]

In [60]:
import pandas as pd
from enum import Enum
from itertools import combinations
from src.utils.enum import get_enum_values
import numpy as np

class EPlatform(str, Enum):
    facebook = "facebook"
    tiktok = "tiktok"
    google = "google"

df = pd.read_csv('data/ads_insights.csv')

In [17]:
set(combinations(, r=2))

{(1, 2), (1, 3), (2, 3)}

In [20]:
def get_all_subsets(s: set | list):
    n = len(s)
    subsets = []
    for i in range(n+1):
        i_subsets = combinations(s, i)
        subsets.extend(i_subsets)
    return subsets

In [24]:
platforms = get_enum_values(EPlatform)
platform_combinations = get_all_subsets(platforms)

In [25]:
table = pd.DataFrame(columns=['combination', 'shops', 'budget_split'])

In [66]:
def get_spend_ratio(s: pd.Series, platforms: list):
    ratio = [s[f"{platform}_spend"] / s.full_spend for platform in platforms]
    return ratio

In [92]:
def get_data_by_platform_combination(df: pd.DataFrame, combination: list):
    filter = [True in range(len(df))]
    for platform in platforms:
        if platform in combination:
            filter = filter & (df[f'{platform}_spend'].notna())
        else:
            filter = filter & (df[f'{platform}_spend'].isna())
    filtered = df[filter].copy()
    filtered['full_spend'] = filtered.apply(lambda df: sum([df[f"{platform}_spend"] for platform in combination]), axis=1)
    platforms_without_the_last_one = combination[:len(combination)-1]
    filtered['spend_ratio'] = filtered.apply(lambda df: get_spend_ratio(df, platforms=platforms_without_the_last_one), axis=1)
    mean_ratio = np.mean(filtered['spend_ratio'].tolist(), axis=0)
    print(mean_ratio)
    result = {"combination": '_'.join(combination), "shops": len(filtered)/len(df), "budget_split": list(mean_ratio) + [1-mean_ratio.sum()]}
    return result

In [95]:
for combination in platform_combinations:
    if not len(combination):
        continue
    row = get_data_by_platform_combination(df=df, combination=combination)
    table.loc[len(table), :] = row

[]
[]
[]
[0.82786697]
[0.5786432]
[0.30833483]
[0.50313534 0.13843614]


In [96]:
table

,combination,shops,budget_split
0,0,0,0
1,facebook,0.392339,[1.0]
2,tiktok,0.001014,[1.0]
3,google,0.260922,[1.0]
4,facebook_tiktok,0.001287,"[0.8278669672713745, 0.17213303272862546]"
5,facebook_google,0.338898,"[0.5786431962833312, 0.42135680371666884]"
6,tiktok_google,0.000195,"[0.30833483167965314, 0.6916651683203469]"
7,facebook_tiktok_google,0.005071,"[0.50313533847963, 0.1384361417520539, 0.35842..."


In [86]:
result

{'combination': 'facebook_tiktok',
 'shops': 0.0012876038862226385,
 'budget_split': [0.8278669672713745, 0.17213303272862546]}

In [78]:
filtered

,shop_id,facebook_spend,year_month,tiktok_spend,google_spend,full_spend,spend_ratio
6739,21740924,2149.28,2023-02,0.00,NaN,2149.28,[1.0]
6750,21740924,9481.70,2023-01,1614.22,NaN,11095.92,[0.854521301523443]
6765,21740924,14873.73,2022-12,50.00,NaN,14923.73,[0.9966496311578942]
7987,28794880,1612.01,2022-12,266.35,NaN,1878.36,[0.8582007708852403]
11383,38348391,1334.41,2022-06,0.00,NaN,1334.41,[1.0]
14907,45057901,130.28,2022-05,103.86,NaN,234.14,[0.556419236354318]
14909,45057901,17532.91,2022-06,1661.25,NaN,19194.16,[0.9134502369470714]
14910,45057901,85947.00,2022-07,168.15,NaN,86115.15,[0.9980473819066681]
14911,45057901,113013.44,2022-12,21792.67,NaN,134806.11,[0.8383406360438708]
15415,45867058,15679.49,2022-02,2511.31,NaN,18190.80,[0.8619461486025903]


In [59]:
filtered.spend_ratio

6739                                      [1.0, 0.0]
6750        [0.854521301523443, 0.14547869847655703]
6765      [0.9966496311578942, 0.003350368842105822]
7987       [0.8582007708852403, 0.14179922911475973]
11383                                     [1.0, 0.0]
14907         [0.556419236354318, 0.443580763645682]
14909      [0.9134502369470714, 0.08654976305292861]
14910    [0.9980473819066681, 0.0019526180933319148]
14911      [0.8383406360438708, 0.16165936395612923]
15415      [0.8619461486025903, 0.13805385139740967]
15416      [0.9231396578706262, 0.07686034212937376]
15420       [0.865440545232338, 0.13455945476766196]
15421       [0.885701633088106, 0.11429836691189399]
15423      [0.8416309889713892, 0.15836901102861078]
15424       [0.8747700613965933, 0.1252299386034067]
15427      [0.8770213078879504, 0.12297869211204959]
15428      [0.9213810167105383, 0.07861898328946171]
15430                                     [1.0, 0.0]
15431     [0.9754048333588794, 0.0245951666411

In [54]:
filtered

,shop_id,facebook_spend,year_month,tiktok_spend,google_spend,full_spend,spend_ratio
6739,21740924,2149.28,2023-02,0.00,NaN,2149.28,[1.0]
6750,21740924,9481.70,2023-01,1614.22,NaN,11095.92,[0.854521301523443]
6765,21740924,14873.73,2022-12,50.00,NaN,14923.73,[0.9966496311578942]
7987,28794880,1612.01,2022-12,266.35,NaN,1878.36,[0.8582007708852403]
11383,38348391,1334.41,2022-06,0.00,NaN,1334.41,[1.0]
14907,45057901,130.28,2022-05,103.86,NaN,234.14,[0.556419236354318]
14909,45057901,17532.91,2022-06,1661.25,NaN,19194.16,[0.9134502369470714]
14910,45057901,85947.00,2022-07,168.15,NaN,86115.15,[0.9980473819066681]
14911,45057901,113013.44,2022-12,21792.67,NaN,134806.11,[0.8383406360438708]
15415,45867058,15679.49,2022-02,2511.31,NaN,18190.80,[0.8619461486025903]


In [52]:
combination[:len(combination)-1]

('facebook',)

In [43]:
len(df[(df.facebook_spend.notna()) & (df.google_spend.isna()) & (df.tiktok_spend.notna())])

33

In [2]:
df = ping_ads_insights_all_platforms(db=db, columns='spend')

In [19]:
only_shop = df.groupby('shop_id').sum().reset_index()

In [3]:
df

,shop_id,spend,year_month
0,51064739,250.00,2022-11
1,45867058,2511.31,2022-02
2,50602673,2413.71,2022-12
3,50944062,2444.15,2022-08
4,47445704,100.11,2021-03
...,...,...,...
157,50944062,140.10,2022-02
158,50742348,20.00,2021-09
159,50692333,34.32,2022-12
160,50742348,478.01,2022-04


In [17]:
FacebookAd.ad_id.__mapper__

AttributeError: Neither 'InstrumentedAttribute' object nor 'Comparator' object associated with FacebookAd.ad_id has an attribute '__mapper__'

In [15]:
FacebookAd.__class__

sqlalchemy.orm.decl_api.DeclarativeMeta

In [3]:
df = ping_shops(db=db)

2023-02-04 08:25:50.218 | DEBUG    | src.pingers.ping_shops:ping_shops:22 - Here I am!


ping_shops: 0:00:02.613746


In [6]:
df.loc[df.shop_id == 16038, :]

,shop_id,name
359,16038,shopfw.myshopify.com


In [4]:
df.ad_id

0                       1
1                       1
2                      10
3       23842588968860342
4       23842589378750342
              ...        
5228    23851164951750755
5229    23851164978180755
5230    23851249585430755
5231    23851314362320755
5232    23851314425550755
Name: ad_id, Length: 5233, dtype: object

In [8]:
df.loc[df.ad_id == '10'].index[0]

2

In [2]:
query = session.query(FacebookAd.ad_id)

In [3]:
query[0]

('23848754996430295',)

In [4]:
type(query)

sqlalchemy.orm.query.Query